In [1]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [9]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_10Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [10]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [11]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [12]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, _= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt':_,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [13]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [14]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response, payload_data['prompt']
    except Exception as e:
        print(e)
        

In [ ]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    
    global df
    for i in range(10):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-68e3eebda7ab47aa89c36181462c164e","object":"text_completion","created":454632,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 3.179744 seconds 


Prompt==> Problem: Write a function in python to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s. 
 {"id":"cmpl-39426f24bbbf4baea13bd2a90edc10d2","object":"text_completion","created":454633,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/c

Prompt==> Problem: Write a function in python to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number. 
 {"id":"cmpl-d5a3805a1a8a4b76bb362c05b9a2fde0","object":"text_completion","created":454633,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nFibonacci sequence: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, ...\n\nThe Fibonacci sequence is defined as follows:\n\nF(0) = 0, F(1) = 1\n\nFor n > 1, F(n) = F(n-1) + F(n-2)\n\nHere is a Python function that generates the nth Fibonacci number:\n\n```python\ndef fibonacci(n):\n    if n <= 0:\n        return \"Input should be a positive integer.\"\n    elif n == 1:\n        return 0\n    elif n == 2:\n        return 1\n    else:\n        a, b = 0, 1\n       ","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":43,"total_tokens":243,"completion_tokens":200}}
Execution time: 11.426511 

Prompt==> Problem: Write a function in java to transpose a given matrix.Input: 2D list representing a matrix.Output: 2D list, the transpose of the input matrix. 
 {"id":"cmpl-4a6c0273687942a19a454f52e3552c90","object":"text_completion","created":454643,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nTranspose of a matrix is obtained by changing rows to columns and columns to rows.\n\nExample:\nInput:\n1 2 3\n4 5 6\n7 8 9\n\nOutput:\n1 4 7\n2 5 8\n3 6 9\n\nHere is a Java function that transposes a given matrix:\n\n```java\npublic class Main {\n    public static void main(String[] args) {\n        int[][] matrix = {{1, 2, 3}, {4, 5, 6}, {7, 8, 9}};\n        int[][] transpose = transposeMatrix(matrix);\n        printMatrix(transpose);\n    }\n\n    public static int[][] transposeMatrix(int[][] matrix) {\n        int rows = matrix.length;\n        int cols = matrix[0].length;","logprobs":null,"f

Prompt==> Problem: Write a function in javascript to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-2d6e00daf7f44f0f81056acd7c82752a","object":"text_completion","created":454658,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nfunction isPalindrome(s) {\n    let reversed = s.split('').reverse().join('');\n    return s === reversed;\n}\n\nconsole.log(isPalindrome('madam')); // true\nconsole.log(isPalindrome('hello')); // false","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":39,"total_tokens":109,"completion_tokens":70}}
Execution time: 4.112178 seconds 


Prompt==> Problem: Write a function in c++ to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-691136795cdf4e6c989e1b00860dec5f","object":"text_completion","crea

Prompt==> Problem: Write a function in javascript to check if a given number is a prime number.Input: Positive integer num.Output: Boolean, indicating whether num is prime. 
 {"id":"cmpl-be903230b030464489296ec0b17582b6","object":"text_completion","created":454658,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nfunction isPrime(num) {\n    if (num <= 1) {\n        return false;\n    }\n    if (num === 2) {\n        return true;\n    }\n    if (num % 2 === 0) {\n        return false;\n    }\n    for (let i = 3; i <= Math.sqrt(num); i += 2) {\n        if (num % i === 0) {\n            return false;\n        }\n    }\n    return true;\n}\n\nconsole.log(isPrime(2)); // true\nconsole.log(isPrime(4)); // false\nconsole.log(isPrime(7)); // true\nconsole.log(isPrime(10)); // false\nconsole.log(isPrime(13)); // true\nconsole.log(isPrime(15)); // false","logprobs":null,"finish_reason":"stop"}],"usag

...

Prompt==> Problem: Write a function in Golang to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-a27b2851e915444da4aa2f830e5baf86","object":"text_completion","created":455306,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```go\npackage main\n\nimport (\n\t\"fmt\"\n\t\"strings\"\n)\n\nfunc isPalindrome(s string) bool {\n\ts = strings.ToLower(s)\n\ti, j := 0, len(s)-1\n\tfor i < j {\n\t\tif s[i] != s[j] {\n\t\t\treturn false\n\t\t}\n\t\ti++\n\t\tj--\n\t}\n\treturn true\n}\n\nfunc main() {\n\tfmt.Println(isPalindrome(\"A man, a plan, a canal: Panama\")) // true\n\tfmt.Println(isPalindrome(\"race a car\")) // false\n}\n```\n\nThis function works by comparing the first and last characters of the string, then the second and second-to-last characters, and so on. If at any point the characters don't match","logprobs":nu

Prompt==> Problem: Build a function in CSS to toggle between light and dark modes for a website.Input: None.Output: None (update the website theme). 
 {"id":"cmpl-c24b797f11ad4d2286608f0438c31293","object":"text_completion","created":455317,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple CSS function to toggle between light and dark modes:\n\n```css\n@media (prefers-color-scheme: dark) {\n  body {\n    background-color: #333;\n    color: #fff;\n  }\n}\n\n@media (prefers-color-scheme: light) {\n  body {\n    background-color: #fff;\n    color: #333;\n  }\n}\n```\n\nThis CSS function uses the `prefers-color-scheme` media feature to detect the user's preferred color scheme. If the user's preferred color scheme is dark, the body background color will be set to a dark grey (#333) and the text color will be set to white (#fff). If the user's preferred color scheme is light, the 

Prompt==> Problem: Create a function in CSS to toggle a specific class on an HTML element when a button is clicked.Input: HTML element selector, class name.Output: None (update the class of the HTML element). 
 {"id":"cmpl-4d5d183bef8d4f8fa1f339d7433f73df","object":"text_completion","created":455324,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple CSS function that toggles a class on an HTML element when a button is clicked:\n\n```css\n@keyframes toggleClass {\n  0% {\n    opacity: 0;\n  }\n  100% {\n    opacity: 1;\n  }\n}\n\nbutton {\n  animation: toggleClass 2s;\n}\n\nbutton:hover {\n  background-color: #f0f0f0;\n}\n\n.myClass {\n  color: red;\n}\n\nbutton:active + .myClass {\n  color: blue;\n}\n```\n\nIn this example, when the button is clicked, the class \"myClass\" is toggled on the HTML element. The animation makes the button fade in over 2 seconds. The hover effect 

Prompt==> Problem: Build a function in react to populate an HTML table with data from an array of objects.Input: Array of objects, each representing a row in the table.Output: None (update the HTML table). 
 {"id":"cmpl-bac3d4d2954b48bc9f4150eebb7c58b6","object":"text_completion","created":455332,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can do this in React:\n\n```jsx\nimport React from 'react';\n\nclass Table extends React.Component {\n  constructor(props) {\n    super(props);\n    this.state = {\n      data: [\n        {id: 1, name: 'John', age: 25},\n        {id: 2, name: 'Jane', age: 30},\n        {id: 3, name: 'Bob', age: 20}\n      ]\n    };\n  }\n\n  render() {\n    return (\n      <table>\n        <thead>\n          <tr>\n            <th>ID</th>\n            <th>Name</th>\n            <th>Age</th>\n          </tr>\n        </thead>\n      

Prompt==> Problem: Create an animated dropdown menu in angular that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu). 
 {"id":"cmpl-c78f79ffee5845888762eaddf129c9f7","object":"text_completion","created":455341,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nSolution:\n\nTo create an animated dropdown menu in Angular, you can use Angular's built-in animations feature. Here's a simple example:\n\nFirst, you need to import the BrowserAnimationsModule in your app.module.ts file:\n\n```typescript\nimport { BrowserAnimationsModule } from '@angular/platform-browser/animations';\n\n@NgModule({\n  imports: [\n    BrowserAnimationsModule\n  ],\n  // ...\n})\nexport class AppModule { }\n```\n\nThen, you can define the animation in your component's animations array:\n\n```typescript\nimport { trigger, state, style,

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))